### Import packages

In [1]:
from urllib import request
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

## Data manipulation - OVERALL LEVEL

In [2]:
df_overall=pd.read_csv('df_overall.csv')
#convert to ints
df_overall['for']=df_overall['for'].astype(int, copy=True, errors='raise')
df_overall.imod=df_overall.imod.astype(int, copy=True, errors='raise')
df_overall.hverkeneller=df_overall.hverkeneller.astype(int, copy=True, errors='raise')
df_overall.fravaer=df_overall.fravaer.astype(int, copy=True, errors='raise')
df_overall.total=df_overall.total.astype(int, copy=True, errors='raise')

In [3]:
df_overall['fullvote'] = np.where(df_overall['total']==179,1,0)
df_overall['inbetween'] = np.where((df_overall['total']<179) & (df_overall['total']>0),1,0)
df_overall['givenvotes'] = df_overall['for']+df_overall['imod']+df_overall['hverkeneller']
df_overall['tally'] = df_overall['for']+df_overall['imod']+df_overall['hverkeneller']+df_overall['fravaer']

df_overall['zerovotes'] = np.where(df_overall['total']==0,1,0)
df_overall2=df_overall[df_overall['zerovotes']==0]
laws_to_be_removed=df_overall[df_overall['zerovotes']==1]
laws_to_be_removed=laws_to_be_removed[['unqiueid','zerovotes','url']]

df_overall2['temp'] = df_overall2.metadata_number.apply(lambda x: (re.findall('(\d{4})',x)))
df_overall2.loc[:, 'year'] = df_overall2.temp.map(lambda x: x[0])

df_overall2['temp'] = df_overall2.metadata_number.apply(lambda x: (re.findall('\d{4}-(\d\d)-\d\d',x)))
df_overall2.loc[:, 'month'] = df_overall2.temp.map(lambda x: x[0])

df_overall2['temp'] = df_overall2.metadata_number.apply(lambda x: (re.findall('\d{4}-\d\d-(\d\d)',x)))
df_overall2.loc[:, 'day'] = df_overall2.temp.map(lambda x: x[0])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [4]:
conditions = [
    (df_overall2['for'] == 0) & (df_overall2['imod'] == 0) & (df_overall2['hverkeneller'] == 0),
    (df_overall2['for'] == 0) & (df_overall2['imod'] == 0) & (df_overall2['hverkeneller'] > 0),
    (df_overall2['for'] == 0) & (df_overall2['imod'] > 0) & (df_overall2['hverkeneller'] == 0),
    (df_overall2['for'] > 0) & (df_overall2['imod'] == 0) & (df_overall2['hverkeneller'] == 0),
    ]
choices = ['agree_allfravear', 'agree_allhverkeneller', 'agree_allimod','agree_allfor']
df_overall2['partyalignment'] = np.select(conditions, choices, default='disagree')

df_overall2[df_overall2['partyalignment']=='disagree']
df_overall2['disagree'] = np.where(df_overall2['partyalignment']=='disagree',1,0)
df_overall2['obs']=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [5]:
df_overall_summed = df_overall2.groupby(['zerovotes','year']).sum()
df_overall_summed2=df_overall_summed.reset_index()
df_overall_summed2['pct_obstain'] = (df_overall_summed2.hverkeneller)/ (df_overall_summed2.givenvotes) * 100
df_overall_summed2['pct_disagree'] = (df_overall_summed2.disagree) / (df_overall_summed2.obs) * 100
df_overall_summed2['pct_fravaer'] = (df_overall_summed2.fravaer) / (df_overall_summed2.total) *100

In [6]:
df_overall_summed = df_overall2.groupby(['samling','year','month']).sum()
df_overall_summed.to_csv('df_overall_summed.csv')
df_party_summed = df_party.groupby(['samling','year','month']).sum()
df_party_summed.to_csv('df_party_summed.csv')

NameError: name 'df_party' is not defined

### Stemmer 'hverken eller'
Ingen udtalt tendens i antal af afgivne stemmer der er hverken eller

In [ ]:
ax1 = sns.barplot(x="year", y="pct_obstain", data=df_overall_summed3)
ax1.set_title('Stemmer "Hverken for eller imod" (Pct)')

## Fravær 
Svag opadgående tendens

In [ ]:
ax2 = sns.barplot(x="year", y="pct_fravaer", data=df_overall_summed3)
ax2.set_title('Fravear (Pct)')

# Uenighed på tværs af partier

In [ ]:
ax3 = sns.barplot(x="year", y="pct_disagree", data=df_overall_summed3)
ax3.set_title('Disagree (Pct)')

# Data manipulation - party level

In [ ]:
df_party=pd.read_csv('df_party.csv')

In [ ]:
#convert to ints
df_party['for']=df_party['for'].astype(int, copy=True, errors='raise')
df_party.imod=df_party.imod.astype(int, copy=True, errors='raise')
df_party.hverkeneller=df_party.hverkeneller.astype(int, copy=True, errors='raise')
df_party.fravaer=df_party.fravaer.astype(int, copy=True, errors='raise')
df_party.total=df_party.total.astype(int, copy=True, errors='raise')

In [ ]:
df_party['temp'] = df_party.metadata_number.apply(lambda x: (re.findall('(\d{4})',x)))
df_party.loc[:, 'year'] = df_party.temp.map(lambda x: x[0])

df_party['temp'] = df_party.metadata_number.apply(lambda x: (re.findall('\d{4}-(\d\d)-\d\d',x)))
df_party.loc[:, 'month'] = df_party.temp.map(lambda x: x[0])

df_party['temp'] = df_party.metadata_number.apply(lambda x: (re.findall('\d{4}-\d\d-(\d\d)',x)))
df_party.loc[:, 'day'] = df_party.temp.map(lambda x: x[0])

In [ ]:
conditions = [
    (df_party['for'] == 0) & (df_party['imod'] == 0) & (df_party['hverkeneller'] == 0),
    (df_party['for'] == 0) & (df_party['imod'] == 0) & (df_party['hverkeneller'] > 0),
    (df_party['for'] == 0) & (df_party['imod'] > 0) & (df_party['hverkeneller'] == 0),
    (df_party['for'] > 0) & (df_party['imod'] == 0) & (df_party['hverkeneller'] == 0),
    ]
choices = ['agree_allfravear', 'agree_allhverkeneller', 'agree_allimod','agree_allfor']
df_party['partyalignment'] = np.select(conditions, choices, default='disagree')

df_party[df_party['partyalignment']=='disagree']
df_party['disagree'] = np.where(df_party['partyalignment']=='disagree',1,0)
df_party['obs']=1

# Subsection where smaller parties are removed

In [ ]:
conditions = [
    (df_party['party']=='IA'),
    (df_party['party']=='JF'),
    (df_party['party']=='UFG'),
    (df_party['party']=='TF'),
    (df_party['party']=='T'),
    (df_party['party']=='SIU'),
    (df_party['party']=='NY'),
    (df_party['party']=='NQ'),
    (df_party['party']=='FF'),
    (df_party['party']=='SP'),
    ]

choices = ['delete', 'delete', 'delete','delete','delete','delete','delete','delete','delete','delete']
df_party['next'] = np.select(conditions, choices, default='ok')
df_party2=df_party[df_party.next=='ok']
#df_party2['year'] = df_party2.samling.apply(lambda x: (re.findall('....',x))[0])

In [ ]:
df_party_summed = df_party2.groupby(['party'])['for','imod','hverkeneller','fravaer','total','disagree', 'obs'].sum()
df_party_summed['share_pct_disagree']=df_party_summed.disagree / df_party_summed.obs * 100
df_party_summed['share_pct_agree']=100-df_party_summed.share_pct_disagree
df_party_summed['share_pct_absence']=df_party_summed.fravaer / df_party_summed.total * 100
df_party_summed['share_pct_obstain']=df_party_summed.hverkeneller / (df_party_summed.total-df_party_summed.fravaer) * 100
df_party_summed #NB check how to exclude parties
df_party_summed=df_party_summed.reset_index()

In [ ]:
df_party_summed=df_party_summed.sort_values(['share_pct_obstain'], ascending=0)

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
ax = sns.barplot(x='party', y='share_pct_obstain', data=df_party_summed)
ax.set_ylabel('')
ax.set_xlabel('Partier')
ax.set_title('Procent af afgivne stemmer der er "hverken for eller imod"')
ax.legend()

### Party - including all parties

In [ ]:
df_party_summed_year = df_party.groupby(['party','year'])['for','imod','hverkeneller','fravaer','total','disagree', 'obs'].sum()
df_party_summed_year['share_pct_disagree']=df_party_summed_year.disagree / df_party_summed_year.obs * 100
df_party_summed_year['share_pct_agree']=100-df_party_summed_year.share_pct_disagree
df_party_summed_year['share_pct_absence']=df_party_summed_year.fravaer / df_party_summed_year.total * 100
df_party_summed_year['share_pct_obstain']=df_party_summed_year.hverkeneller / (df_party_summed_year.total-df_party_summed_year.fravaer) * 100
df_party_summed_year #NB check how to exclude parties
df_party_summed_year=df_party_summed_year.reset_index()
df_party_summed_year.head()

In [ ]:
df_party_summed_year_month = df_party.groupby(['party','year','month'])['for','imod','hverkeneller','fravaer','total','disagree', 'obs'].sum()
#df_party_summed_year = df_party2.groupby(['party','year'])['for','imod','hverkeneller','fravaer','total','disagree', 'obs'].sum()
df_party_summed_year_month['share_pct_disagree']=df_party_summed_year_month.disagree / df_party_summed_year_month.obs * 100
df_party_summed_year_month['share_pct_agree']=100-df_party_summed_year_month.share_pct_disagree
df_party_summed_year_month['share_pct_absence']=df_party_summed_year_month.fravaer / df_party_summed_year_month.total * 100
df_party_summed_year_month['share_pct_obstain']=df_party_summed_year_month.hverkeneller / (df_party_summed_year_month.total-df_party_summed_year_month.fravaer) * 100
df_party_summed_year_month #NB check how to exclude parties
df_party_summed_year_month=df_party_summed_year_month.reset_index()
df_party_summed_year_month
df_party_summed_year_month.to_csv('df_party_summed_year_month.csv')

In [ ]:
df_party_summed_year_month.head()

In [ ]:
temp=df_party.groupby(['unqiueid']).sum()
temp.total.value_counts()

# member analysis

In [ ]:
#Create a dataset we want to append on
datemerge=df_overall2[['unqiueid','year','month','day','zerovotes', 'samling']]

In [ ]:
df_member=pd.read_csv('df_member.csv')

In [ ]:
#convert to ints
df_member['for']=df_member['for'].astype(int, copy=True, errors='raise')
df_member.imod=df_member.imod.astype(int, copy=True, errors='raise')
df_member.hverkeneller=df_member.hverkeneller.astype(int, copy=True, errors='raise')
df_member.fravaer=df_member.fravaer.astype(int, copy=True, errors='raise')
df_member.total=df_member.total.astype(int, copy=True, errors='raise')
df_member.samling=df_member.samling.astype(int, copy=True, errors='raise')

In [ ]:
df_member2=pd.merge(df_member, datemerge, on='unqiueid', how='inner')

In [ ]:
df_member2['yearmonth'] = df_member2['year']+df_member2['month']

In [ ]:
df_member2.yearmonth=df_member2.yearmonth.astype(int, copy=True, errors='raise')

In [ ]:
df_member3=df_member[df_member2.yearmonth>201506]
df_member_summed = df_member3.groupby(['name','party'])['for','imod','hverkeneller','fravaer','total'].sum()
df_member_summed.to_csv('df_member_summed.csv')